In [2]:
cd ..
cd ..

c:\Users\Tanner\Documents\repos\ironmarch


In [3]:
import torch
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from scipy.spatial import distance

In [4]:
latent_dim = 16
n_neighbors = 10
embedding_path = 'saved_embeddings\\vae\sinkhorn\\alpha_1.0_latent_16_softmax_True_version_1.1.2_features_True.csv'

In [6]:
embeddings = pd.read_csv(embedding_path).to_dict(orient = 'records')

def convert_string_to_tensor(string):
    return np.array(string.strip('[').strip(']').split()).astype('float')

embeddings = [{'id' : item['id'], 'embedding' : convert_string_to_tensor(item['embedding'])} for item in embeddings]

embedding_array = []
id_list = []
for embedding in embeddings:
    embedding_array.append(embedding['embedding'])
    id_list.append(embedding['id'])
embedding_array = np.array(embedding_array)
print(embedding_array.shape)

(21715, 16)


In [7]:
archetypes = np.diag(np.array([1.0 for i in range(latent_dim)]))
all_points = np.concatenate([embedding_array, archetypes])
print(all_points.shape)

(21731, 16)


In [8]:
distance_matrix = distance.squareform(distance.pdist(all_points))
archetype_distance = distance_matrix[-16:]
archetype_closest = np.argsort(archetype_distance, axis = 1)
print(archetype_closest)

[[21715  6370   394 ... 21727 21728 21730]
 [21716   690   986 ... 21715 21722 21730]
 [21717 16733 19560 ... 21729 21722 21730]
 ...
 [21728 21470 13486 ... 21729 21721 21730]
 [21729 15436  7586 ... 21715 21721 21730]
 [21730  6382  4983 ... 21729 21722 21718]]


In [9]:
raw_posts = pd.read_csv('iron_march_201911\core_message_posts.csv')

In [10]:
output_directory = 'post_training_analysis/closest_posts/output'
output_file_name = embedding_path.split('\\')[-1].replace('.csv', '') + '.txt'
output_path = os.path.join(output_directory, output_file_name)

with open(output_path, 'w+', encoding = 'utf-8') as output_file:
    for num, archetype in enumerate(archetype_closest):
        output_file.write(f'---- ARCHETYPE {num} ----\n\n')
        for index in archetype[1: 1 + n_neighbors]:
            id = id_list[index]
            mask = raw_posts['msg_id'].values == id
            post = raw_posts.loc[mask]['msg_post'].values[0]
            output_file.write(f'MESSAGE {id}:\n{post.strip()}\n\n')